## Metropolis Hastings Custom HMC MC Algorithm

In [1]:
from IPython.display import display
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
import pandas as pd
import arviz
from multiprocessing import Pool

np.set_printoptions(formatter={'float': lambda x: "{0:0.5f}".format(x)})
plt.style.use('ggplot')
%matplotlib inline

In [ ]:

### no loop ###

def unnormalized_log_prob(x):
    return -x - x**2.

# Initialize the HMC transition kernel.
num_results = int(1e4)
num_burnin_steps = int(1e3)
adaptive_hmc = tfp.mcmc.NoUTurnSampler(
        target_log_prob_fn=unnormalized_log_prob,
        step_size=1.)

# Run the chain (with burn-in).
@tf.function
def run_chain():
    # Run the chain (with burn-in).
    samples, is_accepted = tfp.mcmc.sample_chain(
          num_results=num_results,
          num_burnin_steps=num_burnin_steps,
          current_state=1.,
          kernel=adaptive_hmc,
          trace_fn=lambda _, pkr: pkr.is_accepted)

    sample_mean = tf.reduce_mean(samples)
    sample_stddev = tf.math.reduce_std(samples)
    is_accepted = tf.reduce_mean(tf.cast(is_accepted, dtype=tf.float32))
    return sample_mean, sample_stddev, is_accepted

sample_mean, sample_stddev, is_accepted = run_chain()

print('mean:{:.4f}  stddev:{:.4f}  acceptance:{:.4f}'.format(
    sample_mean.numpy(), sample_stddev.numpy(), is_accepted.numpy()))


In [ ]:
# %%time

### scalar sent through ###
def unnormalized_log_prob(x):
    return -x - x**2.

# Initialize the HMC transition kernel.
num_results = int(1e4)
num_burnin_steps = int(1e3)
adaptive_hmc = tfp.mcmc.NoUTurnSampler(
        target_log_prob_fn=unnormalized_log_prob,
        step_size=1.)

# Run the chain (with burn-in).
@tf.function
def run_chain(val):
#     tf.print('here', val)
    # Run the chain (with burn-in).
    return tfp.mcmc.sample_chain(
          num_results=1,
          num_burnin_steps=0,
          current_state=val,
          kernel=adaptive_hmc,
          trace_fn=lambda _, pkr: pkr.is_accepted)

    
sample_list = list()
sample_list.append([1.])
for i in range(num_results):
    samples, is_accepted = run_chain(sample_list[i][0])
    sample_list.append(samples)
    


In [ ]:
samples = np.array(sample_list)
print(samples.shape)
sample_mean = tf.reduce_mean(samples)
sample_stddev = tf.math.reduce_std(samples)
is_accepted = tf.reduce_mean(tf.cast(is_accepted, dtype=tf.float32))


print('mean:{:.4f}  stddev:{:.4f}  acceptance:{:.4f}'.format(
    sample_mean.numpy(), sample_stddev.numpy(), is_accepted.numpy()))
